**Задание 1**

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [3]:
import pandas as pd

In [5]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [22]:
result = ratings.groupby('userId').agg({'rating':'count', 'timestamp':['max','min']})
result.head()

rating   timestamp            
        count         max         min
userId                               
1          20  1260759205  1260759108
2          76   835356246   835355395
3          51  1298932787  1298861589
4         204   949982274   949778714
5         100  1163375145  1163373044

In [28]:
result = result[result['rating']['count'] > 100]

In [30]:
result['diff'] = result['timestamp']['max'] -result['timestamp']['min']
result

rating   timestamp                   diff
        count         max         min           
userId                                          
4         204   949982274   949778714     203560
8         116  1154474527  1154389340      85187
15       1700  1469330735   997937239  471393496
17        363  1127476640  1127468587       8053
19        423   855195373   855190091       5282
...       ...         ...         ...        ...
656       128   986244044   986240991       3053
659       142   866207451   834598040   31609411
664       519  1441911722  1343731283   98180439
665       434  1046967769   992836298   54131471
671       115  1074784735  1063500751   11283984

[258 rows x 4 columns]

In [31]:
result['diff'].mean()

40080507.4496124

**Задание 2**

Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия):   
- rzd - железнодорожные перевозки
- auto - автомобильные перевозки
- air - воздушные перевозки
- client_base - адреса клиентов  

Необходимо сформировать две таблицы:  
   
-таблицу с тремя типами выручки для каждого client_id без указания адреса клиента;   
-аналогичную таблицу по типам выручки с указанием адреса клиента.   
    
Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [65]:
(
    client_base[['client_id']].merge(rzd, on = 'client_id', how = 'outer')
    .merge(auto, on = 'client_id', how = 'outer')
    .merge(air, on = 'client_id', how = 'outer')
)

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,57483.0,NaN
3,114,5774.0,83.0,NaN
4,115,981.0,912.0,81.0
5,116,NaN,4834.0,4.0
6,117,NaN,98.0,13.0
7,118,NaN,NaN,173.0


In [66]:
(
    client_base.merge(rzd, on = 'client_id', how = 'outer')
    .merge(auto, on = 'client_id', how = 'outer')
    .merge(air, on = 'client_id', how = 'outer')
)

,client_id,address,rzd_revenue,auto_revenue,air_revenue
0,111,Комсомольская 4,1093.0,NaN,NaN
1,112,Энтузиастов 8а,2810.0,NaN,NaN
2,113,Левобережная 1а,10283.0,57483.0,NaN
3,114,Мира 14,5774.0,83.0,NaN
4,115,ЗЖБИиДК 1,981.0,912.0,81.0
5,116,Строителей 18,NaN,4834.0,4.0
6,117,Панфиловская 33,NaN,98.0,13.0
7,118,Мастеркова 4,NaN,NaN,173.0


In [36]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [37]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [38]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [39]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


**Задание 3**

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:  

У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
   
Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?   
    
Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

1. Исходя из имеющихся данных по каждому user_id могу предположить, что помимо информации о последовательности
координат (широта/долгота) в таблице имеется дополнительная аналитика отражающая время, которую в дальнейшем
можно сначала использовать в качестве ключа при объединении с таблицей визиты, а потом и таблицей покупки.

2. Если исходить из того, что вторым показателем в таблице координат является значение времени
в данной точке координат, то значительное удаление координат друг от друга на объединения таблиц не окажет.

3. Из информации о координатах пользователя и время нахождения в них можно сделать вывод, каким местам он отдает больше
предпочтения и как это могло повлиять на решения связанные с приобритением того или иного продукта.